## Set Up

In [20]:
#import from python modules:
import cPickle as pickle
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq

import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
with open ('../data/cleaned_df_24NOV17.pkl', 'rb') as f:
    df = pickle.load(f)

## Create New DF with Event Count and Repeat Data for each Person ID

In [22]:
person_event_count_df = df.groupby(by='PersonID').count()

In [23]:
df.groupby(by='PersonID').count().index.values

array([    1,     2,     3, ..., 11669, 11670, 11671])

In [24]:
df.groupby(by='PersonID').count()['EventID'].values

array([1, 2, 1, ..., 1, 1, 1])

In [25]:
repeat_attendees_df = pd.DataFrame.from_dict({'PersonID': person_event_count_df.index.values, 
                                              'Event_Count': person_event_count_df['EventID'].values})

In [26]:
repeat_attendees_df.head()

,Event_Count,PersonID
0,1,1
1,2,2
2,1,3
3,1,4
4,2,5


## Pull and Format Data from Baseline DF to Add to New Repeat Attendees DF

* Event IDs
* Event Dates
* Series IDs
* Calculate lag time between first and second events for those who have attended more than one event ('Repeat Lag Time')
* ID whether a person is one off, repeat within a single series, or repeat across multiple series ('Cross_Series_Attend')

In [27]:
personID_eventID_dict = defaultdict(list)
personID_eventdate_dict = defaultdict(list)
personID_seriesID_dict = defaultdict(list)

for num in person_event_count_df.index.values:
    personID_eventID_dict[num].append(list(df[df['PersonID']==num]['EventID'].values))
    personID_eventdate_dict[num].append(list(df[df['PersonID']==num]['Event_Date'].values))
    personID_seriesID_dict[num].append(list(df[df['PersonID']==num]['SeriesID'].values))

In [28]:
repeat_attendees_df['Event_IDs'] = repeat_attendees_df['PersonID'].map(personID_eventID_dict).apply(lambda x: x[0])
repeat_attendees_df['Event_Dates'] = repeat_attendees_df['PersonID'].map(personID_eventdate_dict).apply(lambda x: x[0])
repeat_attendees_df['Series_IDs'] = repeat_attendees_df['PersonID'].map(personID_seriesID_dict).apply(lambda x: x[0])

In [30]:
def date_trunc(date_list):
    return [np.datetime64(item, 'D') for item in date_list]

In [31]:
repeat_attendees_df['Event_Dates'] = repeat_attendees_df['Event_Dates'].apply(lambda x: date_trunc(x))

In [32]:
def repeat_lag_time(person_ID, duration_days=183):
    if person_ID in repeat_attendees_df['PersonID'].values:
        date_list = sorted(repeat_attendees_df[repeat_attendees_df['PersonID'] == person_ID]['Event_Dates'].values[0])
        if len(date_list) == 1:
            return 'One Off'
        elif len(date_list) > 1:
            first_event, second_event = heapq.nsmallest(2, date_list)
            delta = second_event - first_event
            if delta < np.timedelta64(duration_days, 'D'):
                return 'Under {} days'.format(duration_days)
            else:
                return 'Over {} days'.format(duration_days)
    else:
        print 'Provided PersonID does not exist in the dataframe'

In [33]:
repeat_attendees_df['Repeat_Lag_Time'] = repeat_attendees_df['PersonID'].apply(lambda x: repeat_lag_time(x))

In [34]:
repeat_attendees_df['Cross_Series_Attend'] = repeat_attendees_df['Series_IDs'].apply(lambda x: 
                                            'One Off' if len(x) == 1 else
                                            ('Single - {}'.format(x[0]) if len(set(x)) == 1 else 'Multiple'))

In [35]:
repeat_attendees_df.head()

,Event_Count,PersonID,Event_IDs,Event_Dates,Series_IDs,Repeat_Lag_Time,Cross_Series_Attend
0,1,1,[11],[2015-02-15],[2.0],One Off,One Off
1,2,2,"[9, 17]","[2015-01-24, 2015-06-09]","[1.0, 3.0]",Under 183 days,Multiple
2,1,3,[30],[2015-11-07],[1.0],One Off,One Off
3,1,4,[18],[2015-06-20],[2.0],One Off,One Off
4,2,5,"[13, 40]","[2015-03-14, 2016-04-16]","[0.0, 2.0]",Over 183 days,Multiple


## Get Counts for Single/Repeat Attendees by Type (One Series vs. Multi Series)

In [36]:
total_repeats_count = float(len(repeat_attendees_df[repeat_attendees_df['Event_Count']>1]))
total_repeats_count

2739.0

In [37]:
multi_attend_single_series = repeat_attendees_df[repeat_attendees_df['Cross_Series_Attend'] > 'Single']

In [38]:
single_series = float(len(multi_attend_single_series))
single_series

1241.0

In [39]:
multi_attend_multi_series = repeat_attendees_df[repeat_attendees_df['Cross_Series_Attend'] == 'Multiple']

In [40]:
multi_series = len(multi_attend_multi_series)
multi_series

1498

In [41]:
single_attend = repeat_attendees_df[repeat_attendees_df['Cross_Series_Attend'] == 'One Off']

In [42]:
single_attend_count = len(single_attend)
single_attend_count

8693

In [43]:
total_people = float(len(repeat_attendees_df))
total_people

11432.0

In [44]:
'{}% of attendees are repeat. {}% of repeat attendees have only attended one series \
and {}% have attended multiple'.format(round((single_series+multi_series)/total_people*100), 
                                       round(single_series/total_repeats_count * 100), 
                                       round(multi_series/total_repeats_count * 100))

'24.0% of attendees are repeat. 45.0% of repeat attendees have only attended one series and 55.0% have attended multiple'

In [45]:
multi_attend_single_series_counts = multi_attend_single_series.groupby(by='Cross_Series_Attend')['Event_Count'].count()

In [46]:
multi_attend_single_series_counts.index.values

array(['Single - 0.0', 'Single - 1.0', 'Single - 2.0', 'Single - 3.0',
       'Single - 4.0'], dtype=object)

In [47]:
print 'For the {} Repeat Attendees all in One Series, Breakdown is:'.format(int(single_series))
for idx, val in enumerate(multi_attend_single_series_counts):
    print 'Series {}: {}%'.format(idx, round(val/single_series * 100))

For the 1241 Repeat Attendees all in One Series, Breakdown is:
Series 0: 3.0%
Series 1: 24.0%
Series 2: 45.0%
Series 3: 10.0%
Series 4: 18.0%


## Create New DF Capturing Repeat Attendee Data by Event ID

In [ ]:
event_person_count_df = df.groupby(by='EventID').count()

In [ ]:
events_df = pd.DataFrame.from_dict({'EventID': event_person_count_df.index.values, 
                                              'Total_Part_Count': event_person_count_df['PersonID'].values})

In [ ]:
eventID_personID_dict = defaultdict(list)
eventID_eventdate_dict = defaultdict(list)
eventID_eventname_dict = defaultdict(list)
eventID_seriesID_dict = defaultdict(list)
eventdate_dict = defaultdict(list)
eventname_dict = defaultdict(list)
seriesID_dict = defaultdict(list)

for num in event_person_count_df.index.values:
    eventID_personID_dict[num].append(list(df[df['EventID']==num]['PersonID'].values))
    eventID_eventdate_dict[num].append(list(df[df['EventID']==num]['Event_Date'].values))
    eventID_eventname_dict[num].append(list(df[df['EventID']==num]['Event_Name'].values))
    eventID_seriesID_dict[num].append(list(df[df['EventID']==num]['SeriesID'].values))
    
    eventdate_dict[num] = eventID_eventdate_dict[num][0][0]
    eventname_dict[num] = eventID_eventname_dict[num][0][0]
    seriesID_dict[num] = eventID_seriesID_dict[num][0][0]

In [ ]:
events_df['Person_IDs'] = events_df['EventID'].map(eventID_personID_dict).apply(lambda x: x[0])
events_df['Event_Date'] = events_df['EventID'].map(eventdate_dict).apply(lambda x: np.datetime64(x, 'D'))
events_df['Event_Name'] = events_df['EventID'].map(eventname_dict)
events_df['Series_ID'] = events_df['EventID'].map(seriesID_dict)

In [ ]:
event_data_dict = defaultdict(list)
for event in events_df['EventID'].values:
    repeats_singleseries = 0
    repeats_multiseries = 0
    single_attend = 0
    first_timer = 0
    under_six_month_repeat = 0
    over_six_month_repeat = 0
    first_timer_ids = list()
    for person in events_df[events_df['EventID'] == event]['Person_IDs'].values[0]:
        att_status = repeat_attendees_df[repeat_attendees_df['PersonID'] == person]['Cross_Series_Attend'].values[0]
        lag_status = repeat_attendees_df[repeat_attendees_df['PersonID'] == person]['Repeat_Lag_Time'].values[0]
        if att_status == 'Multiple':
            repeats_multiseries += 1
        elif att_status == 'One Off':
            single_attend += 1
        else:
            repeats_singleseries += 1
            
        if min(repeat_attendees_df[repeat_attendees_df['PersonID']==person]['Event_Dates'].values[0]) == events_df[
            events_df['EventID']==event]['Event_Date'].values[0]:
            first_timer += 1
            first_timer_ids.append(person)
            if lag_status == 'Under 183 days':
                under_six_month_repeat += 1
            elif lag_status == 'Over 183 days':
                over_six_month_repeat += 1
            
    event_data_dict['repeat_singleseries'].append(repeats_singleseries)
    event_data_dict['repeat_multiseries'].append(repeats_multiseries)
    event_data_dict['single_attend'].append(single_attend)
    event_data_dict['first_timer'].append(first_timer)
    event_data_dict['six_mo_repeat'].append(under_six_month_repeat)
    event_data_dict['longer_repeat'].append(over_six_month_repeat)
    event_data_dict['first_timer_ids'].append(first_timer_ids)

In [ ]:
events_df['Repeat_SingleSeries'] = event_data_dict['repeat_singleseries']
events_df['Repeat_MultiSeries'] = event_data_dict['repeat_multiseries']
events_df['Single_Attend'] = event_data_dict['single_attend']
events_df['First_Timers'] = event_data_dict['first_timer']
events_df['First_Timer_IDs'] = event_data_dict['first_timer_ids']
events_df['Repeat_Under_Six_Mo'] = event_data_dict['six_mo_repeat']
events_df['Repeat_Over_Six_Mo'] = event_data_dict['longer_repeat']

In [ ]:
events_df.sort_values(by='Event_Date', ascending=True, inplace=True)

In [ ]:
events_df['%_Repeat_Under_Six'] = 100 * events_df['Repeat_Under_Six_Mo'] / (events_df['First_Timers'])
events_df['%_Repeat_Under_Six'] = events_df['%_Repeat_Under_Six'].apply(lambda x: int(x))

events_df['%_Repeat_Over_Six'] = 100 * events_df['Repeat_Over_Six_Mo'] / (events_df['First_Timers'])
events_df['%_Repeat_Over_Six'] = events_df['%_Repeat_Over_Six'].apply(lambda x: int(x))

In [ ]:
events_df.reset_index(drop=True, inplace=True)

In [ ]:
events_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

X = events_df['Event_Date'].iloc[0:80].values #include data through June 7th 2017; 6 months ago
Y = events_df['%_Repeat_Over_Six'].iloc[0:80].values + events_df['%_Repeat_Under_Six'].iloc[0:80].values
Names = events_df['Event_Name'].iloc[0:80].values
Series = events_df['Series_ID'].iloc[0:80].values
EventID = events_df['EventID'].iloc[0:80].values

ax.plot(X,Y);
ax.set_title('% of First Timers at Each Event That Have Returned at Any Time', size=20);

ax.hlines(y = 50, xmin=events_df['Event_Date'][0], xmax=events_df['Event_Date'][80], color='r');

#for x,y,n,s, e in zip(X, Y, Names, Series, EventID):
#    ax.annotate(s = '{}, Series {}, Event {}, {}%'.format(n, s, e, y), xy=(x,y), xytext=(x,y), size=15)

### Event Instances with High % of First Timers Returning Ever ( > 50%):

#### 2015:
* Absolution Run, ID 8, 50%, Winter Series
* Cedar Mountain Trail Run, ID 13, 65%, No Series
* Ravenna Refresher, ID 31, 53%, Winter Series

#### 2016:
* Woodland Park Zoom, ID 43, 55%, Summer Series
* Wilburton Hilbilly, ID 44, 74%, Summer Series
* Sunshine Salutation, ID 48, 76%, Summer Series

#### 2017:
* None to date

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

X = events_df['Event_Date'].iloc[0:80].values #include data through June 7th 2017; 6 months ago
Y = events_df['%_Repeat_Under_Six'].iloc[0:80].values
Names = events_df['Event_Name'].iloc[0:80].values
Series = events_df['Series_ID'].iloc[0:80].values
EventID = events_df['EventID'].iloc[0:80].values

ax.plot(X,Y);
ax.set_title('% of First Timers at Each Event That Return Within 6 Months', size=20);

ax.hlines(y = 30, xmin=events_df['Event_Date'][0], xmax=events_df['Event_Date'][80], color='r');

#for x,y,n,s, e in zip(X, Y, Names, Series, EventID):
#    ax.annotate(s = '{}, {}, {}, {}%'.format(n, s, e, y), xy=(x,y), xytext=(x,y), size=15)

### Event Instances with High % of First Timers Returning in Next 6 Months ( > 30%):

#### 2015:
* Absolution Run, ID 8, 39%, Winter Series
* Cedar Mountain Trail Run, ID 13, 45%, No Series
* Ravenna Run the Ravine, ID 17, 34%, Summer Series
* Summer Eddy, ID 23, 36%, Summer Series
* Carkeek Cooler, ID 30, 39%, Winter Series
* Ravenna Refresher, ID 31, 42%, Winter Series

#### 2016:
* Woodland Park Zoom, ID 43, 45%, Summer Series
* Wilburton Hilbilly, ID 44, 58%, Summer Series
* Sunshine Salutation, ID 48, 64%, Summer Series
* Ravenna Refresher, ID 60, 40%, Winter Series

#### 2017:
* Woodland Park Zoom, ID 73, 33%, Summer Series
* Wilburton Hilbilly, ID 75, 35%, Summer Series

In [ ]:
sum(events_df['First_Timers'].iloc[0:80].values) / float(sum(events_df['Total_Part_Count'].iloc[0:80].values)) * 30

In [ ]:
#if repeat the above plot but normalize for number of first timers relative to size of event:

fig, ax = plt.subplots(figsize=(24,12))

X = events_df['Event_Date'].iloc[0:80].values #include data through June 7th 2017; 6 months ago
Y = events_df['%_Repeat_Under_Six'].iloc[0:80].values * events_df['First_Timers'].iloc[0:80].values / events_df['Total_Part_Count'].iloc[0:80].values
Names = events_df['Event_Name'].iloc[0:80].values
Series = events_df['Series_ID'].iloc[0:80].values
EventID = events_df['EventID'].iloc[0:80].values

ax.plot(X,Y);
ax.set_title('% of First Timers at Each Event That Return Within 6 Months', size=20);

ax.hlines(y = 20, xmin=events_df['Event_Date'][0], xmax=events_df['Event_Date'][80], color='r');

#for x,y,n,s, e in zip(X, Y, Names, Series, EventID):
#    ax.annotate(s = '{}, {}, {}, {}%'.format(n, s, e, y), xy=(x,y), xytext=(x,y), size=15)

### Event Instances with High % of First Timers Returning in Next 6 Months When Normalized for Number of First Timers at Event ( > 20%):

#### 2015:
* Absolution Run, ID 8, 38%, Winter Series
* Interlaken Icicle Dash, ID 9, 23%, Winter Series
* Cedar Mountain Trail Run, ID 13, 25%, No Series
* Ravenna Run the Ravine, ID 17, 24%, Summer Series
* Carkeek Cooler, ID 30, 27%, Winter Series
* Ravenna Refresher, ID 31, 23%, Winter Series

Drops Out: Summer Eddy, ID 23, Summer Series

#### 2016:
* Sunshine Salutation, ID 48, 22%, Summer Series
* Ravenna Refresher, ID 60, 21%, Winter Series

Drops Out: Woodland Park Zoom, ID 43, Summer Series; Wilburton Hillbilly, ID 44, Summer Series

#### 2017:
* None meet 20% threshold

Drops Out: Woodland Park Zoom, ID 73, Summer Series; Wilburton Hilbilly, ID 75, Summer Series

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))

X = events_df['Event_Date'].iloc[0:80].values #include data through June 7th 2017; 6 months ago
Y = events_df['%_Repeat_Over_Six'].iloc[0:80].values
Names = events_df['Event_Name'].iloc[0:80].values
Series = events_df['Series_ID'].iloc[0:80].values

ax.plot(X,Y);
ax.set_title('% of First Timers at Each Event That Return After Over 6 Months', size=20);

#ax.hlines(y = 30, xmin=events_df['Event_Date'][0], xmax=events_df['Event_Date'][80], color='r')

for x,y,n,s in zip(X, Y, Names, Series):
    ax.annotate(s = '{}, {}, {}%'.format(n, s, y), xy=(x,y), xytext=(x,y))

### Look at an Individual Event with High Rate of Return of First Timers:

In [ ]:
events_df[events_df['EventID']==48]

In [ ]:
def zero_to_nan(values):
    """Replace every 0 with 'nan' and return a copy."""
    return [np.nan if x==0 else x for x in values]

In [ ]:
def single_event_future_repeats(event_id):
    '''Create dataframe for counts of when first timers at a given event have their first repeat attendance, based on 
    perspective of event with provided event_id'''
    idx = (events_df[events_df['EventID']==event_id].index.values[0]) + 1
    event_person_list = events_df[events_df['EventID']==event_id]['First_Timer_IDs'].values[0]
    future_events_df = events_df[['Event_Name', 'EventID', 'Event_Date', 'Series_ID']].iloc[idx:]
    already_repeated_list = list()
    event_future_repeats_list = list()
    
    for event in future_events_df['EventID'].values:
        repeat_att = 0
        for person in events_df[events_df['EventID']==event]['Person_IDs'].values[0]:
            if person in event_person_list and person not in already_repeated_list:
                repeat_att += 1
                already_repeated_list.append(person)
        event_future_repeats_list.append(repeat_att)
    
    future_events_df['Repeats'] = zero_to_nan(event_future_repeats_list)
    
    return future_events_df

In [ ]:
def plot_single_event_future_repeats(event_id_list):
    
    subplot_size = len(event_id_list)
    fig, axes = plt.subplots(subplot_size, 1, figsize=(20,30))
    
    for ax, event_id in zip(axes.flatten(), event_id_list):
        event_futurerepeats_df = single_event_future_repeats(event_id)
        X = event_futurerepeats_df['Event_Date'].values
        Y = event_futurerepeats_df['Repeats'].values
        ymax = np.nanmax(Y) + np.nanmax(Y) / 2
        ymin = np.nanmin(Y) / 10
        Names = event_futurerepeats_df['Event_Name'].values
        Series = event_futurerepeats_df['Series_ID'].values
        
        ax.scatter(X,Y);
        ax.set_title('Spread of First Repeat for First Timers at Event #{}, {}'.format(
            event_id,events_df[events_df['EventID']==event_id]['Event_Name'].values[0]), size=13);
        ax.set_xlabel('Event Date')
        ax.set_ylabel('Number of Attendees')
        ax.set_ylim((ymin, ymax))
        
        for x,y,n,s in zip(X, Y, Names, Series):
            ax.annotate(s = '{}'.format(n), xy=(x,y), xytext=(x,y), verticalalignment='bottom', 
                       horizontalalignment = 'left', rotation = 60)
            
        textstr = 'Total Event Size: {} \n First Timers: {}'.format(
            events_df[events_df['EventID']==event_id]['Total_Part_Count'].values[0], 
            events_df[events_df['EventID']==event_id]['First_Timers'].values[0], 
            events_df[events_df['EventID']==event_id])
        
        #these are matplotlib.path.Patch properties
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        
        #place a text box in the upper left in axes coords
        ax.text(0.85, 0.96, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)

In [ ]:
plot_single_event_future_repeats([43, 44, 48, 60])